In [0]:
import keras

from keras.layers import Input, Dense, Lambda, Flatten, Reshape, BatchNormalization, Activation, Dropout, Conv2D, Conv2DTranspose
from keras.optimizers import Adam
from keras.models import Model
from keras import metrics
from keras import backend as K

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit
import pickle
import os
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)


work_DIR = "/content/drive/My Drive/Deep Learning Course/project_301558086_305143315/YOLO3_Barcode_Detection/qqwweee2"

from os import chdir
chdir(work_DIR)


Mounted at /content/drive


#find center#

In [0]:
import numpy as np
from PIL import Image
import os
directory_in_str = work_DIR+'/BW_BB'
directory = os.fsencode(directory_in_str)
num_pic = len(os.listdir(directory))
filepath_arr = []
short_filepath_arr = []
image_filepath_arr = []
arr= np.empty([num_pic,480,640,4])
i=0
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".png"):
        img = Image.open(work_DIR+'/BW_BB/'+filename).convert('RGBA')
        
        if img.size == (640,480):
          arr[i] = np.array(img)
          filepath_arr.append(work_DIR+'/BW_BB/'+filename)
          short_filepath_arr.append('/BW_BB/'+filename) # readable
          image_filepath_arr.append('/pictures/'+filename) 
          i = i+1
          
        
        continue
    else:
        continue
print(arr.shape)
array=arr[0:i-1,:,:,:3]
filepath_array = filepath_arr[0:i-1]
print(array.shape)

(365, 480, 640, 4)
(207, 480, 640, 3)


In [0]:
#print(arr[1])
height = 480
rows = np.zeros([207,480]) # 1 for with white, -1 for all black
cols = np.zeros([207,640]) # 1 for with white, -1 for all black
edges = np.zeros([207,4], dtype=int) #top bot left right
xywh_array = np.zeros([207,4])
for j in range(207): #  pic j
  #print(j)
  for h in range(480): # height index k
    for w in range(640): # w index 
      if array[j,h,w,1] == 255. and edges[j,0] == 0: # first white in line h
        edges[j,0] = int(h)#/480
        
        rows[j,h] = 1
        break
      elif array[j,h,w,1] == 255.:
        rows[j,h] = 1
        break
      else:
        rows[j,h] = -1
    if rows[j,h] == -1 and rows[j,h-1] == 1: # h-1 was last whit row      
      edges[j,1] = int(h-1)#/480
      break
  for w in range(640):
    for h in range(480):
      if array[j,h,w,1] == 255. and edges[j,2] == 0: # first white in col w
        edges[j,2] = int(w)#/640
        cols[j,w] = 1
        break # kleave col
      elif array[j,h,w,1] == 255.:
        cols[j,w] = 1 #white in col
        break
      else:
        cols[j,w] = -1
    if cols[j,w] == -1 and cols[j,w-1] == 1: # w-1 was last white col      
      edges[j,3] = int(w-1)#/640
      
  xywh_array[j,2] = (edges[j,3]-edges[j,2])
  xywh_array[j,3] = (edges[j,1]-edges[j,0])
  xywh_array[j,0] = (edges[j,2]+edges[j,3])/(2)
  xywh_array[j,1] = (edges[j,0]+edges[j,1])/(2)

Create the txt file

In [0]:
with open('train.txt', 'a') as the_file:
  for pic in range(207):
    
    #the_file.write(filepath_array[pic]+" %s,%s,%s,%s,0 \n" % (xywh_array[pic,0],xywh_array[pic,1],xywh_array[pic,2],xywh_array[pic,3]))
     #the_file.write(short_filepath_arr[pic]+" %s,%s,%s,%s,0 \n" % (edges[pic,2],edges[pic,0],edges[pic,1],edges[pic,3]))
      the_file.write(image_filepath_arr[pic]+" %s,%s,%s,%s,0 \n" % (edges[pic,2],edges[pic,0],edges[pic,1],edges[pic,3]))

#Cutouts of existing pictures#

```
# This is formatted as code
```



In [0]:
import numpy as np
from PIL import Image
with open(work_DIR+'/train.txt', 'r') as the_file:
  lines = the_file.readlines()
  
  pics_num = len(lines)
  for pic in range(pics_num):
    
    line = lines[pic].split(" ")
    try:
      image = Image.open(work_DIR+line[0]).convert('RGBA') 
    except:
      try:
        image = Image.open((work_DIR+line[0]).replace("png", "jpg")).convert('RGBA')
      except:
        try:
          image = Image.open((work_DIR+line[0]).replace("jpg", "png")).convert('RGBA')
        except:
          print("can't open image")
          raise Exception("can't open image")

    pixels = np.array(image)
    xmin,ymin,ymax,xmax,class_num = line[1].split(',')
    xmin = int(xmin)
    xmax = int(xmax)
    ymin = int(ymin)
    ymax = int(ymax)
    
    new_img = pixels[ymin:ymax,xmin:xmax]
    pic_name = line[0].replace("/pictures/", "",1)
    cut_image = Image.fromarray(new_img)
    cut_image.save(work_DIR+"/cutted_images/"+pic_name)
    del image
print('done')   

<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.

#covert weights to keras#

In [0]:
!python convert.py yolov3.cfg yolov3.weights model_data/yolo_weights.h5